In [1]:
from config import DATA_DIR
from utils import sample_series, save_df, read_df
import os
import pandas as pd
import nltk
import re

Abrindo os dados originais parseados

In [2]:
df: pd.DataFrame = read_df('dados_orginais_contribuicoes.csv')

In [3]:
df.sample(3)

,Unnamed: 0,id,canal,texto,temas
1691,1691,2428,Participe+,Implantação da Estratégia Saúde da Família (ES...,['Saúde']
1373,1373,2094,Participe+,Meta 40 - Iniciativa C - Revisar - Faz-se nece...,['Urbanismo e Licenciamento']
2191,2191,2987,Audiência Regional - Ermelino Matarazzo,"Boa noite, eu participo da frente democrática ...","['Cultura', 'Zeladoria Urbana e melhorias de b..."


In [4]:
df.drop('Unnamed: 0', axis=1, inplace=True)

Limpezas simples do texto -- tirar maiusculas etc.

In [5]:
df.rename({'texto' : 'texto_original'}, axis=1, inplace=True)

In [6]:
df['texto_limpo'] = df['texto_original'].str.lower().str.strip('/n')


In [7]:
print(sample_series(df['texto_limpo']))

meta 37 - iniciativa j - revisar - incluir a implantação de faixa elevada de pedestre ao longo da calçada verde da rua manuel dos reis araújo no jardim marajoara para melhoria da segurança dos munícipes que praticam caminhadas e diversas outras atividades físicas e de lazer no local.


Baixando as stopwords para limpar os textos

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/h-pgy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [10]:
stopwords[:3]

['a', 'à', 'ao']

In [11]:
type(stopwords)

list

In [12]:
pdm_stopwords: list[str] = ['programa de metas',
                            'pdm',
                            'audiência',
                            'audiencia',
                            'boa noite',
                            'bom dia',
                            'boa tarde',
                            'meta',
                            'olá',
                            'iniciativa'
                        ]

In [13]:
stopwords.extend(pdm_stopwords)

stopwords = set(stopwords)

tokenizando o texto

In [14]:
from nltk.tokenize import word_tokenize

In [15]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/h-pgy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
df['tokens'] = df['texto_limpo'].apply(word_tokenize)

In [17]:
print(sample_series(df['tokens']))

['meta', '37', '-', 'iniciativa', 'j', '-', 'revisar', '-', 'incluir', 'a', 'implantação', 'de', 'faixa', 'elevada', 'de', 'pedestre', 'ao', 'longo', 'da', 'calçada', 'verde', 'da', 'rua', 'manuel', 'dos', 'reis', 'araújo', 'no', 'jardim', 'marajoara', 'para', 'melhoria', 'da', 'segurança', 'dos', 'munícipes', 'que', 'praticam', 'caminhadas', 'e', 'diversas', 'outras', 'atividades', 'físicas', 'e', 'de', 'lazer', 'no', 'local', '.']


In [18]:
def remove_stopwords(tokens:str, stopwords:set[str]=stopwords)->list[str]:

    filtered: list[str]=[token for token in tokens
                         if token not in stopwords]
    
    return filtered





In [19]:
df['tokens_clean'] = df['tokens'].apply(remove_stopwords)

In [20]:
print(sample_series(df['tokens_clean']))

['37', '-', 'j', '-', 'revisar', '-', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local', '.']


Rmover os números - são referência às metas, podem enviesar o modelo.

In [21]:
'214'.isdigit()

True

In [22]:
def remove_numbers(tokens:list[str])->list[str]:

    return [token for token in tokens
            if not token.isdigit()]

In [23]:
df['tokens_clean'] = df['tokens_clean'].apply(remove_numbers)

In [24]:
print(sample_series(df['tokens_clean']))

['-', 'j', '-', 'revisar', '-', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local', '.']


Remover caracteres especiais e pontuacao

In [25]:
def remover_char_especiais(tokens:list[str])->list[str]:

    #tudo o que não são caracteres alphanumericos
    patt = r'[^\w]'
    limpo:list[str] = []
    for token in tokens:
        token = re.sub(patt, '', token)
        if token:
            limpo.append(token)
    return limpo

In [26]:
remover_char_especiais(['ontem', '123', '@'])

['ontem', '123']

In [27]:
df['tokens_clean'] = df['tokens_clean'].apply(remover_char_especiais)

In [28]:
print(sample_series(df['tokens_clean']))

['j', 'revisar', 'incluir', 'implantação', 'faixa', 'elevada', 'pedestre', 'longo', 'calçada', 'verde', 'rua', 'manuel', 'reis', 'araújo', 'jardim', 'marajoara', 'melhoria', 'segurança', 'munícipes', 'praticam', 'caminhadas', 'diversas', 'outras', 'atividades', 'físicas', 'lazer', 'local']


In [29]:
save_df(df, 'dados_tokenizados.csv')

'../data/dados_tokenizados.csv'